In [ ]:
# ===================================
# Section 1: Installation of Libraries and Imports
# ===================================

# Install required libraries (run once in Google Colab)
!pip install ipywidgets gensim

# Download necessary data from NLTK
import nltk
nltk.download('punkt_tab')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')  # Added for wordnet

# ===================================
# Section 2: Main Libraries Import
# ===================================

# Libraries for natural language processing and data analysis
import numpy as np
from collections import Counter, defaultdict
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

# Gensim library for building topic modeling like LDA, TF-IDF
from gensim import corpora, models, matutils
from gensim.similarities import Similarity

# Libraries for plotting graphs
import matplotlib.pyplot as plt
import seaborn as sns

# Libraries for creating interactive widgets in Jupyter Notebook
import ipywidgets as widgets
from IPython.display import display

# Libraries for data table handling
import pandas as pd

# Libraries for text similarity measurements
from sklearn.metrics.pairwise import cosine_similarity
from scipy.spatial.distance import euclidean, cityblock
from sklearn.preprocessing import normalize

# ===================================
# Section 3: Preprocessing Functions
# ===================================

def preprocess_text(text):
    """
    Function to prepare text for use
    """
    try:
        tokens = word_tokenize(text)  # Tokenize text into words
        lower_tokens = [t.lower() for t in tokens]  # Convert all words to lowercase
        alpha_only = [t for t in lower_tokens if t.isalpha()]  # Remove non-alphabetic tokens
        no_stops = [t for t in alpha_only if t not in stopwords.words('english')]  # Remove stopwords
        wordnet_lemmatizer = WordNetLemmatizer()  # Create lemmatizer object
        words_lemmatized = [wordnet_lemmatizer.lemmatize(t) for t in no_stops]  # Lemmatize words
        return words_lemmatized
    except Exception as e:
        print(f"Error in text preprocessing: {e}")
        # Fallback: use simple split if tokenizer fails
        words = text.lower().split()
        alpha_only = [w for w in words if w.isalpha()]
        return alpha_only

def bow_to_vector(bow, all_words):
    """
    Function to convert Bag-of-Words to a vector based on all words present
    """
    return np.array([bow.get(word, 0) for word in all_words])

# ===================================
# Section 4: Document and Query Processing System
# ===================================

def process_documents(documents, query):
    """
    Function to process all documents against the user's query
    """
    bow_documents = [Counter(preprocess_text(doc)) for doc in documents]
    bow_query = Counter(preprocess_text(query))

    all_words = set(bow_query.keys())
    for doc in bow_documents:
        all_words.update(doc.keys())

    all_words = sorted(all_words)

    vectors = [bow_to_vector(bow, all_words) for bow in bow_documents]
    query_vector = bow_to_vector(bow_query, all_words)

    similarity_scores = cosine_similarity([query_vector], vectors)
    ranked_indices = similarity_scores.argsort()[0][::-1]

    results = [(idx, similarity_scores[0][idx]) for idx in ranked_indices]
    return results

# ===================================
# Section 5: Widget Interface System
# ===================================

def setup_widgets():
    """
    Function to set up all widgets
    """
    global documents, query, upload_widget, query_input

    # Variables to store data
    documents = []
    query = ""

    # Function triggered when files are uploaded
    def on_upload_change(change):
        global documents
        uploaded_files = change['new']
        documents = []
        for name, file in uploaded_files.items():
            try:
                documents.append(file['content'].decode('utf-8'))
            except Exception as e:
                print(f"Error reading file {name}: {e}")
        print(f"Successfully uploaded {len(documents)} files!")

    # Function triggered when user presses Enter in the query input box
    def on_query_submit(change):
        global query
        query = query_input.value
        if not documents:
            print("Please upload documents first!")
            return
        if not query.strip():
            print("Please enter a query!")
            return

        try:
            results = process_documents(documents, query)
            print(f"\nResults for query: '{query}'")
            print("-" * 50)
            for idx, score in results:
                print(f"Document {idx}: Similarity = {score:.4f}")
            print()
        except Exception as e:
            print(f"Error processing query: {e}")

    # Create widgets
    upload_widget = widgets.FileUpload(
        accept='.txt',
        multiple=True,
        description='Upload Files'
    )

    query_input = widgets.Text(
        description='Query:',
        placeholder='Enter your query here'
    )

    # Link event handlers
    upload_widget.observe(on_upload_change, names='value')
    query_input.on_submit(on_query_submit)

    return upload_widget, query_input

# ===================================
# Section 6: Similarity Analysis Functions
# ===================================

def prepare_tfidf_data():
    """
    Function to prepare TF-IDF data for analysis
    """
    global tfidf_corpus, dictionary, documents, query

    if not documents or not query:
        print("Please upload documents and enter a query first!")
        return False

    try:
        # Preprocess all documents first
        texts = [preprocess_text(doc) for doc in documents]
        texts.append(preprocess_text(query))

        # Create dictionary and corpus
        dictionary = corpora.Dictionary(texts)
        corpus = [dictionary.doc2bow(text) for text in texts]

        # Create TF-IDF model
        tfidf_model = models.TfidfModel(corpus)
        tfidf_corpus = tfidf_model[corpus]

        return True
    except Exception as e:
        print(f"Error preparing TF-IDF data: {e}")
        return False

def cosine_similarity_analysis():
    """
    Cosine similarity analysis
    """
    if not prepare_tfidf_data():
        return

    try:
        # Create similarity index
        index = Similarity(output_prefix=None, corpus=tfidf_corpus, num_features=len(dictionary))
        keyword_vector = tfidf_corpus[-1]

        similarities = index[keyword_vector]
        similarities = similarities[:-1]  # Remove the query itself

        print("Cosine Similarity Results:")
        print("-" * 40)
        for i, similarity in enumerate(similarities):
            print(f"Query vs Document {i}: {similarity:.4f}")

        # Plot heatmap
        similarity_matrix = np.array(similarities).reshape(1, -1)

        plt.figure(figsize=(12, 3))
        sns.heatmap(
            similarity_matrix,
            annot=True,
            cmap="coolwarm",
            cbar=True,
            xticklabels=[f'Doc {i}' for i in range(len(similarities))],
            yticklabels=['Query']
        )
        plt.title("Cosine Similarity Heatmap")
        plt.xlabel("Documents")
        plt.ylabel("Query")
        plt.tight_layout()
        plt.show()

    except Exception as e:
        print(f"Error in cosine similarity analysis: {e}")

def euclidean_distance_analysis():
    """
    Euclidean distance analysis
    """
    if not prepare_tfidf_data():
        return

    try:
        def sparse_to_dense(sparse_vector, num_terms):
            dense_vector = np.zeros(num_terms)
            for term_id, weight in sparse_vector:
                dense_vector[term_id] = weight
            return dense_vector

        num_terms = len(dictionary)
        dense_tfidf_corpus = [sparse_to_dense(vector, num_terms) for vector in tfidf_corpus]

        keyword_vector = dense_tfidf_corpus[-1]
        document_vectors = dense_tfidf_corpus[:-1]

        distances = [euclidean(keyword_vector, vector) for vector in document_vectors]

        print("Euclidean Distance Results:")
        print("-" * 40)
        for i, distance in enumerate(distances):
            print(f"Query vs Document {i}: {distance:.4f}")

        # Plot heatmap
        distance_matrix = np.array(distances).reshape(1, -1)

        plt.figure(figsize=(12, 3))
        sns.heatmap(
            distance_matrix,
            annot=True,
            cmap="plasma",
            cbar=True,
            xticklabels=[f'Doc {i}' for i in range(len(distances))],
            yticklabels=['Query']
        )
        plt.title("Euclidean Distance Heatmap")
        plt.xlabel("Documents")
        plt.ylabel("Query")
        plt.tight_layout()
        plt.show()

    except Exception as e:
        print(f"Error in euclidean distance analysis: {e}")

def jaccard_similarity_analysis():
    """
    TF-IDF Weighted Jaccard similarity analysis
    """
    if not prepare_tfidf_data():
        return

    try:
        def tfidf_to_dict(tfidf_vector):
            return dict(tfidf_vector)

        query_vec = tfidf_to_dict(tfidf_corpus[-1])
        doc_vecs = [tfidf_to_dict(vec) for vec in tfidf_corpus[:-1]]

        similarities = []

        for doc_vec in doc_vecs:
            all_keys = set(query_vec.keys()) | set(doc_vec.keys())
            numerator = sum(min(query_vec.get(k, 0), doc_vec.get(k, 0)) for k in all_keys)
            denominator = sum(max(query_vec.get(k, 0), doc_vec.get(k, 0)) for k in all_keys)
            score = numerator / denominator if denominator else 0
            similarities.append(score)

        print("TF-IDF Weighted Jaccard Similarity Results:")
        print("-" * 50)
        for i, sim in enumerate(similarities):
            print(f"Query vs Document {i}: {sim:.4f}")

        # Plot heatmap
        similarity_matrix = np.array(similarities).reshape(1, -1)

        plt.figure(figsize=(12, 3))
        sns.heatmap(
            similarity_matrix,
            annot=True,
            cmap="Blues",
            cbar=True,
            xticklabels=[f'Doc {i}' for i in range(len(similarities))],
            yticklabels=['Query']
        )
        plt.title("Jaccard Similarity Heatmap")
        plt.xlabel("Documents")
        plt.ylabel("Query")
        plt.tight_layout()
        plt.show()

    except Exception as e:
        print(f"Error in jaccard similarity analysis: {e}")

def manhattan_distance_analysis():
    """
    Manhattan distance analysis
    """
    if not prepare_tfidf_data():
        return

    try:
        dense_matrix = matutils.corpus2csc(tfidf_corpus).T.toarray()
        norm_matrix = normalize(dense_matrix, norm='l1')

        query_vec = norm_matrix[-1]
        doc_vectors = norm_matrix[:-1]

        distances = [cityblock(query_vec, doc_vec) for doc_vec in doc_vectors]

        print("Manhattan Distance Results:")
        print("-" * 40)
        for i, distance in enumerate(distances):
            print(f"Query vs Document {i}: {distance:.4f}")

        # Plot heatmap
        distance_matrix = np.array(distances).reshape(1, -1)

        plt.figure(figsize=(12, 3))
        sns.heatmap(
            distance_matrix,
            annot=True,
            cmap="cool",
            cbar=True,
            xticklabels=[f'Doc {i}' for i in range(len(distances))],
            yticklabels=['Query']
        )
        plt.title("Manhattan Distance Heatmap")
        plt.xlabel("Documents")
        plt.ylabel("Query")
        plt.tight_layout()
        plt.show()

    except Exception as e:
        print(f"Error in manhattan distance analysis: {e}")

def run_all_analyses():
    """
    Run all analyses
    """
    print("=" * 60)
    print("TEXT SIMILARITY ANALYSIS RESULTS")
    print("=" * 60)

    cosine_similarity_analysis()
    print("\n" + "=" * 60 + "\n")

    euclidean_distance_analysis()
    print("\n" + "=" * 60 + "\n")

    jaccard_similarity_analysis()
    print("\n" + "=" * 60 + "\n")

    manhattan_distance_analysis()
    print("\n" + "=" * 60)

# ===================================
# Section 8: NLTK Issue Workaround (Backup)
# ===================================

# Backup function in case NLTK has issues
def download_nltk_data():
    """
    Download all required NLTK data
    """
    resources = ['punkt', 'punkt_tab', 'stopwords', 'wordnet', 'omw-1.4']
    for resource in resources:
        try:
            nltk.download(resource, quiet=True)
            print(f"✓ Downloaded {resource}")
        except Exception as e:
            print(f"✗ Failed to download {resource}: {e}")

# Run download immediately
print("Downloading NLTK resources...")
download_nltk_data()
print("NLTK setup complete!")

# ===================================
# Section 7: System Initialization (Edited)
# ===================================

# Set global variables
documents = []
query = ""
tfidf_corpus = None
dictionary = None

# Create and display widgets
print()
print("Text Similarity Analysis System")
print("=" * 40)
print("1. Upload your .txt files using the widget below")
print("2. Enter your query in the text box")
print("3. Use run_all_analyses() to see all similarity metrics")
print()

upload_widget, query_input = setup_widgets()

display(upload_widget)
display(query_input)

print("\nAfter uploading files and entering query, run:")
print("run_all_analyses()")



Results for query: 'data'
--------------------------------------------------
Document 6: Similarity = 0.3897
Document 7: Similarity = 0.1659
Document 10: Similarity = 0.1304
Document 2: Similarity = 0.1057
Document 5: Similarity = 0.0596
Document 4: Similarity = 0.0492
Document 9: Similarity = 0.0304
Document 0: Similarity = 0.0238
Document 8: Similarity = 0.0189
Document 3: Similarity = 0.0000
Document 1: Similarity = 0.0000

✓ Downloaded punkt
✓ Downloaded punkt_tab
✓ Downloaded stopwords
✓ Downloaded wordnet
✓ Downloaded omw-1.4
NLTK setup complete!

Text Similarity Analysis System
1. Upload your .txt files using the widget below
2. Enter your query in the text box
3. Use run_all_analyses() to see all similarity metrics



[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


FileUpload(value={}, accept='.txt', description='Upload Files', multiple=True)

Text(value='', description='Query:', placeholder='Enter your query here')


After uploading files and entering query, run:
run_all_analyses()
Successfully uploaded 11 files!

Results for query: 'computer'
--------------------------------------------------
Document 1: Similarity = 0.5762
Document 5: Similarity = 0.5162
Document 9: Similarity = 0.5060
Document 3: Similarity = 0.3153
Document 2: Similarity = 0.2774
Document 4: Similarity = 0.2461
Document 0: Similarity = 0.1668
Document 6: Similarity = 0.1039
Document 10: Similarity = 0.0797
Document 8: Similarity = 0.0568
Document 7: Similarity = 0.0207


Results for query: 'network'
--------------------------------------------------
Document 8: Similarity = 0.2935
Document 3: Similarity = 0.1802
Document 2: Similarity = 0.1717
Document 10: Similarity = 0.0507
Document 6: Similarity = 0.0455
Document 1: Similarity = 0.0360
Document 5: Similarity = 0.0199
Document 0: Similarity = 0.0119
Document 7: Similarity = 0.0041
Document 9: Similarity = 0.0000
Document 4: Similarity = 0.0000

